In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/test123/samsum-test.csv
/kaggle/input/assignment-no-4nlp/samsum-test.csv
/kaggle/input/prediction/samsum-test-predictions.csv
/kaggle/input/1234test/samsum-train.csv
/kaggle/input/1234test/samsum-test.csv
/kaggle/input/1234test/samsum-validation.csv


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import re
import pickle

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, LayerNormalization, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.train import Checkpoint, CheckpointManager
from tensorflow.keras.metrics import Mean

# Ppreprocessing

In [ ]:
df = pd.read_csv('/kaggle/input/1234test/samsum-train.csv')
df.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [ ]:
dialogue = df['dialogue'].astype(str)
summary = df['summary'].astype(str)

In [ ]:
def add_tokens(x):
    return '<START> ' + x + ' <END>'

summary = summary.apply(add_tokens)
print("Summary after adding tokens:")
print(summary)

Summary after adding tokens:
0        <START> Amanda baked cookies and will bring Je...
1        <START> Olivia and Olivier are voting for libe...
2        <START> Kim may try the pomodoro technique rec...
3        <START> Edward thinks he is in love with Bella...
4        <START> Sam is confused, because he overheard ...
                               ...                        
14727    <START> Romeo is trying to get Greta to add hi...
14728    <START> Theresa is at work. She gets free food...
14729    <START> Japan is going to hunt whales again. I...
14730    <START> Celia couldn't make it to the afternoo...
14731    <START> Georgia and Juliette are looking for a...
Name: summary, Length: 14732, dtype: object


In [ ]:
filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
oov_token = '<unk>'

dia_tokenizer = Tokenizer(oov_token=oov_token)
summary_tokenizer = Tokenizer(filters=filters, oov_token=oov_token)

dia_tokenizer.fit_on_texts(dialogue)
summary_tokenizer.fit_on_texts(summary)

inputs = dia_tokenizer.texts_to_sequences(dialogue)
targets = summary_tokenizer.texts_to_sequences(summary)

In [ ]:
summary_tokenizer.texts_to_sequences(["Hello world"])

[[8416, 1267]]

In [ ]:
summary_tokenizer.sequences_to_texts([[8416, 1267]])

['hello world']

In [ ]:
encoder_vocab_size = len(dia_tokenizer.word_index) + 1
decoder_vocab_size = len(summary_tokenizer.word_index) + 1


encoder_vocab_size, decoder_vocab_size

(38359, 17604)

In [ ]:
dia_lengths = []
summary_lengths = []

for n,s in zip(dialogue, summary):
    dia_lengths.append(len(n))
    summary_lengths.append(len(s))



dia_lengths = pd.Series(dia_lengths)
summary_lengths = pd.Series(summary_lengths)

In [ ]:
dia_lengths.describe(),summary_lengths.describe()

(count    14732.000000
 mean       511.183818
 std        402.621178
 min          3.000000
 25%        216.000000
 50%        401.000000
 75%        694.000000
 max       5492.000000
 dtype: float64,
 count    14732.000000
 mean       124.134673
 std         60.892471
 min         17.000000
 25%         77.000000
 50%        110.000000
 75%        158.000000
 max        314.000000
 dtype: float64)

In [ ]:
encoder_maxlen =700
decoder_maxlen = 160

In [ ]:
inputs = pad_sequences(inputs, maxlen=encoder_maxlen, padding='post', truncating='post')
targets = pad_sequences(targets, maxlen=decoder_maxlen, padding='post', truncating='post')

In [ ]:
inputs = tf.cast(inputs, dtype=tf.int32)
targets = tf.cast(targets, dtype=tf.int32)

In [ ]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
count = 0
for inputs , targets in enumerate(ds):
    print(inputs)
    print(targets)
    count+=1

    if count > 2:
        break

0
(<tf.Tensor: shape=(64, 700), dtype=int32, numpy=
array([[1561,  131,    4, ...,    0,    0,    0],
       [  94,  232,   59, ...,    0,    0,    0],
       [5462,   16,  117, ...,    0,    0,    0],
       ...,
       [ 280,   26,    4, ...,    0,    0,    0],
       [3430,   54,  101, ...,    0,    0,    0],
       [ 246,   26,    4, ...,    0,    0,    0]], dtype=int32)>, <tf.Tensor: shape=(64, 160), dtype=int32, numpy=
array([[    2,  1518,   158, ...,     0,     0,     0],
       [    2,    69,  1020, ...,     0,     0,     0],
       [    2, 16681,     6, ...,     0,     0,     0],
       ...,
       [    2,   248,   552, ...,     0,     0,     0],
       [    2,  9539,     6, ...,     0,     0,     0],
       [    2,   208,     6, ...,     0,     0,     0]], dtype=int32)>)
1
(<tf.Tensor: shape=(64, 700), dtype=int32, numpy=
array([[17415,   368,    10, ...,     0,     0,     0],
       [ 3199,    14,     4, ...,     0,     0,     0],
       [  289,    27,    23, ...,     0,   

# Positional Encoding

In [ ]:
def get_angles(position, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return position * angle_rates

In [ ]:
def positional_encoding(position, d_model):
    position = np.arange(position)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    exponent = (2 * (i // 2)) / np.float32(d_model)
    angle_rates = 1 / np.power(10000, exponent)
    angle_rads = position * angle_rates
    angle_rads[:, ::2] = np.sin(angle_rads[:, ::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = np.expand_dims(angle_rads, axis=0)
    pos_encoding = tf.cast(pos_encoding, dtype=tf.float32)

    return pos_encoding

In [ ]:
def create_padding_mask(seq):
    padding_mask = tf.math.equal(seq, 0)
    padding_mask = tf.cast(padding_mask, tf.float32)

    padding_mask = tf.expand_dims(padding_mask, axis=1)
    padding_mask = tf.expand_dims(padding_mask, axis=2)
    return padding_mask

def create_look_ahead_mask(size):
    ones = tf.ones((size, size))

    req_matrix = tf.linalg.band_part(ones, -1, 0)
    toggle_req_matrix = 1 - req_matrix
    mask = toggle_req_matrix
    return mask

## Scaled Dot Product

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled += (mask * -1e9)

    attention_weights = tf.nn.softmax(scaled, axis=-1)

    values = tf.matmul(attention_weights, v)
    return values, attention_weights

## Multihead Attention

In [ ]:
class MultiHeadAttention(Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads

        self.head_dim = d_model // self.num_heads

        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)

        self.linear_dense = Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, q, k, v, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        values, attention_weights = scaled_dot_product_attention(q, k, v, mask)

        values = tf.transpose(values, perm=[0, 2, 1, 3])

        concat_values = tf.reshape(values, (batch_size, -1, self.d_model))
        output = self.linear_dense(concat_values)

        return output, attention_weights

## Feed Forward Network

In [ ]:
def point_wise_feed_forward_network(d_model, hidden):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(hidden, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

In [ ]:
class PointwiseFeedForward(Layer):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PointwiseFeedForward, self).__init__()
        self.linear1 = Dense(hidden, activation='relu')
        self.linear2 = Dense(d_model)
        self.dropout = Dropout(rate=drop_prob)

    def call(self, x):
        x = self.linear1(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

## Encoder layer

In [ ]:
class EncoderLayer(Layer):
    def __init__(self, d_model, num_heads, hidden, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate=rate)

        self.ffn = PointwiseFeedForward(d_model, hidden, drop_prob=rate)
        self.norm2 = LayerNormalization(epsilon=1e-6)
        self.dropout2 = Dropout(rate=rate)


    def call(self, x, training, mask):
        residual_x = tf.identity(x)
        x, _ = self.attention(x, x, x, mask)
        x = self.dropout1(x, training=training)
        x = self.norm1(residual_x + x)
        residual_x = tf.identity(x)

        x = self.ffn(x)
        x = self.dropout2(x, training=training)
        x = self.norm2(residual_x + x)

        return x

## Decoder layer

In [ ]:
class DecoderLayer(Layer):
    def __init__(self, d_model, num_heads, hidden, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.attention1 = MultiHeadAttention(d_model, num_heads)
        self.attention2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = PointwiseFeedForward(d_model, hidden, drop_prob=rate)

        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.norm2 = LayerNormalization(epsilon=1e-6)
        self.norm3 = LayerNormalization(epsilon=1e-6)

        self.dropout1 = Dropout(rate=rate)
        self.dropout2 = Dropout(rate=rate)
        self.dropout3 = Dropout(rate=rate)


    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        output1, attention_weights1 = self.attention1(x, x, x, look_ahead_mask)
        output1 = self.dropout1(output1, training=training)
        output1 = self.norm1(output1 + x)

        output2, attention_weights2 = self.attention2(output1, enc_output, enc_output,  padding_mask)
        output2 = self.dropout2(output2, training=training)
        output2 = self.norm2(output2 + output1)

        output3 = self.ffn(output2)
        output3 = self.dropout3(output3, training=training)
        output3 = self.norm3(output3 + output2)

        return output3, attention_weights1, attention_weights2

In [ ]:
class Encoder(Layer):
    def __init__(self,  d_model, num_layers, num_heads, hidden, input_vocab_size, max_pos_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(max_pos_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, hidden, rate) for _ in range(num_layers)]

    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training=training, mask=mask)

        return x

In [ ]:
class Decoder(Layer):
    def __init__(self, d_model, num_layers,  num_heads, hidden, target_vocab_size, max_pos_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(max_pos_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, hidden, rate) for _ in range(num_layers)]
        self.dropout = Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training=training,look_ahead_mask=look_ahead_mask, padding_mask=padding_mask)


            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

        return x, attention_weights

In [ ]:
class Transformer(Model):
    def __init__(self, d_model, num_layers, num_heads, hidden, input_vocab_size, target_vocab_size, max_pos_input, max_pos_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(d_model, num_layers, num_heads, hidden, input_vocab_size, max_pos_input, rate)
        self.decoder = Decoder(d_model, num_layers, num_heads, hidden, target_vocab_size, max_pos_target, rate)
        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training=training, mask=enc_padding_mask)
        dec_output, attention_weights = self.decoder(
            tar,
            enc_output,
            training=training,
            look_ahead_mask=look_ahead_mask,
            padding_mask=dec_padding_mask
        )

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights


In [ ]:
num_layers = 4
d_model = 128
hidden = 512
num_heads = 8
EPOCHS = 10

In [ ]:
class CustomSchedule(LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step* (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [ ]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')

In [ ]:
transformer = Transformer(
    d_model,
    num_layers,
    num_heads,
    hidden,
    encoder_vocab_size,
    decoder_vocab_size,
    max_pos_input=encoder_vocab_size,
    max_pos_target=decoder_vocab_size,
)

In [ ]:
def create_masks(inputs, targets):
    enc_padding_mask = create_padding_mask(inputs)
    dec_padding_mask = create_padding_mask(inputs)

    look_ahead_mask = create_look_ahead_mask(tf.shape(targets)[1])

    dec_target_padding_mask = create_padding_mask(targets)

    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask

In [ ]:
checkpoint_path = "checkpoints"

ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)


if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print('Latest checkpoint restored!')

Latest checkpoint restored!


In [ ]:
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(
            inp,
            tar_inp,
            training=True,
            enc_padding_mask=enc_padding_mask,
            look_ahead_mask=combined_mask,
            dec_padding_mask=dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)

In [ ]:
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_state()
    count = 0
    for (batch, (inp, tar)) in enumerate(ds):
        train_step(inp, tar)
        if batch % 429 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, train_loss.result()))
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))

    print ('Epoch {} Loss {:.4f}'.format(epoch + 1, train_loss.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))




/opt/conda/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['embeddings', 'embeddings', 'kernel'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


Epoch 1 Batch 0 Loss 9.5986
Epoch 1 Loss 7.0358
Time taken for 1 epoch: 171.7232985496521 secs

Epoch 2 Batch 0 Loss 6.7087
Epoch 2 Loss 6.2426
Time taken for 1 epoch: 138.20018768310547 secs

Epoch 3 Batch 0 Loss 5.8198
Epoch 3 Loss 5.5727
Time taken for 1 epoch: 138.19334769248962 secs

Epoch 4 Batch 0 Loss 5.1906
Epoch 4 Loss 5.2870
Time taken for 1 epoch: 138.19213342666626 secs

Epoch 5 Batch 0 Loss 4.9891
Saving checkpoint for epoch 5 at checkpoints/ckpt-3
Epoch 5 Loss 5.0799
Time taken for 1 epoch: 139.00749444961548 secs

Epoch 6 Batch 0 Loss 4.9618
Epoch 6 Loss 4.8891
Time taken for 1 epoch: 138.19578433036804 secs

Epoch 7 Batch 0 Loss 4.7513
Epoch 8 Loss 4.5300
Time taken for 1 epoch: 138.19642066955566 secs

Epoch 9 Batch 0 Loss 4.4462
Epoch 9 Loss 4.3510
Time taken for 1 epoch: 138.18839716911316 secs

Epoch 10 Batch 0 Loss 4.0235
Saving checkpoint for epoch 10 at checkpoints/ckpt-4
Epoch 10 Loss 4.1787
Time taken for 1 epoch: 138.99944972991943 secs



In [ ]:
import tensorflow as tf

def evaluate(input_news):
    input_news = dia_tokenizer.texts_to_sequences([input_news])
    input_news = tf.keras.preprocessing.sequence.pad_sequences(input_news, maxlen=encoder_maxlen, padding='post', truncating='post')

    encoder_input = tf.expand_dims(input_news[0], 0)

    decoder_input = [summary_tokenizer.word_index['<start>']]
    output = tf.expand_dims(decoder_input, 0)

    for i in range(decoder_maxlen):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input,
            output,
            training=False,
            enc_padding_mask=enc_padding_mask,
            look_ahead_mask=combined_mask,
            dec_padding_mask=dec_padding_mask
        )

        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == summary_tokenizer.word_index['<end>']:
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights

In [ ]:
def summarize(input_news):
    summarized = evaluate(input_news=input_news)[0].numpy()
    summarized = np.expand_dims(summarized[1:], 0)
    return summary_tokenizer.sequences_to_texts(summarized)[0]

In [ ]:
df = df.drop(columns=['id'])
df.head()

,dialogue,summary
0,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [ ]:
def checkinbulk(randomnumber):
    print('predicted summary:', summarize(df.iloc[randomnumber]['dialogue']))
    print('News: ', df.iloc[randomnumber]['dialogue'])
    print('Actual summary: ', summary[randomnumber][7:-6])

In [ ]:
import random
random_number = random.randint(0, 1000)

checkinbulk(random_number)

predicted summary: the team is looking for a new job in the end of the month the year old company is going to be held in august
News:  John: <file_other>
John: this blog is awesome
Ewan: I know this one
John: <file_other>
John: thats the trip I want to do next year
Ewan: man you need like 3 weeks of holidays :)
John: I can just quit this shitty job ;)
Ewan: <file_other>
Ewan: check out this one
Ewan: the guy is obsessed with the world's most remote islands
Ewan: he's been on south pacific, indian ocean
Ewan: and now waiting for a ship to tristan da cunha
John: I saw a documentary about it on tv
John: amazing place, no escape from there
Ewan: and no chance to find you once you get there ;)
Actual summary:   John wants to go on a trip next year. He and Ewan are following a blog of a person who travels to remote islands.
